# Advanced Event 'Loops'

Iterating in python is relatively slow, though it can be useful for prototyping.

First, lets declare a projection function that takes an event and projects out the neutrino energy from it

In [2]:
import pyNUISANCE as pn

evs = pn.EventSource("dune_argon_sf_10mega.nuwro.pb.gz")
if not evs:
    print("Error: failed to open input file")

# just get the first event so that we can check the units
ev,_ = evs.first()
ToGeV = 1 if (ev.momentum_unit() == pn.hm.HepMC3.Units.GEV) else 1E-3

def enu_GeV(ev):
    bpart = pn.pps.sel.Beam(ev,14)
    if bpart:
        return bpart.momentum().e()*ToGeV
    return -0

print("Found muon neutrino beam particle with %.2f GeV of energy" % enu_GeV(ev))

ModuleNotFoundError: No module named 'pyNUISANCE'

## DataFrames

NUISANCE provides the `FrameGen` facility for defining functional event loops and then executing them in batch. Lets see an example of how it works. We include the `limit` call to stop the internal loop running over the entire file.

In [2]:
print(pn.FrameGen(evs).limit(10).evaluate())

 --------------
 | evt# | cvw |
 --------------
 |    0 |   1 |
 |    1 |   1 |
 |    2 |   1 |
 |    3 |   1 |
 |    4 |   1 |
 |    5 |   1 |
 |    6 |   1 |
 |    7 |   1 |
 |    8 |   1 |
 |    9 |   1 |
 --------------


### New Columns
The Frame returned from `FrameGen.evaluate` always contains the event number and the CV weight for all processed events. These are a useful start, but we can define a new column to hold the neutrino energy for each event.

In [3]:
print(pn.FrameGen(evs).limit(10).add_column("enu_GeV",enu_GeV).evaluate())

 ------------------------
 | evt# | cvw | enu_GeV |
 ------------------------
 |    0 |   1 |   2.275 |
 |    1 |   1 |    14.3 |
 |    2 |   1 |    2.86 |
 |    3 |   1 |   3.728 |
 |    4 |   1 |    9.08 |
 |    5 |   1 |   3.237 |
 |    6 |   1 |   2.473 |
 |    7 |   1 |   1.916 |
 |    8 |   1 |   1.988 |
 |    9 |   1 |   3.671 |
 ------------------------


## Filters

We can apply filters in a similar way to the batched loop

In [16]:
print(pn.FrameGen(evs).limit(50).filter(lambda x : not (x.event_number() % 5)).add_column("enu_GeV",enu_GeV).evaluate())

 ------------------------
 | evt# | cvw | enu_GeV |
 ------------------------
 |    0 |   1 |   2.275 |
 |    5 |   1 |   3.237 |
 |   10 |   1 |   2.506 |
 |   15 |   1 |   2.682 |
 |   20 |   1 |   1.528 |
 |   25 |   1 |   3.214 |
 |   30 |   1 |   3.033 |
 |   35 |   1 |   2.014 |
 |   40 |   1 |   14.87 |
 |   45 |   1 |   1.661 |
 ------------------------


### A Short Race

Lets see if there is any appreciable difference in the looping speed between a python loop and a C++ loop:

In [4]:
import time

time_start = time.perf_counter()
for i, (ev, cvw) in enumerate(evs):
    if i >= 1E6:
        break
    enu_GeV(ev)
time_end = time.perf_counter()
print("event loop took %.2fs" % (time_end-time_start))

event loop took 26.44s


In [5]:
time_start = time.perf_counter()
pn.FrameGen(evs).limit(int(1E6)).add_column("enu_GeV",enu_GeV).evaluate()
time_end = time.perf_counter()
print("FrameGen took %.2fs" % (time_end-time_start))

FrameGen took 29.38s


So that it actually slower than doing it directly in python! Probably due to the overheads of calling the python function from the C++ side.

### ProSelecta

We can use ProSelecta to write and JIT C++ functions and then use the JITted functions to create new columns

In [1]:
pn.pps.load_text("""
double enu_GeV(HepMC3::GenEvent const &ev){
  auto bpart = ps::sel::Beam(ev,14);
  if(bpart) return bpart->momentum().e()*%f;
  return -0;
}
""" % ToGeV)
enu_GeV_cpp = pn.pps.project.enu_GeV

NameError: name 'pn' is not defined

In [7]:
time_start = time.perf_counter()
pn.FrameGen(evs).limit(int(1E6)).add_column("enu_GeV",enu_GeV_cpp).evaluate()
time_end = time.perf_counter()
print("FrameGen with ProSelecta took %.2fs" % (time_end-time_start))

FrameGen with ProSelecta took 23.75s


Not much faster... probably could do with some profiling and optimization

## Progress
We can ask FrameGen to report it's progress as it goes

In [12]:
print(pn.FrameGen(evs).limit(int(1E6)).progress(int(1E6)-1).add_column("enu_GeV",enu_GeV_cpp).evaluate())

 ------------------------
 | evt# | cvw | enu_GeV |
 ------------------------
 |    0 |   1 |   2.275 |
 |    1 |   1 |    14.3 |
 |    2 |   1 |    2.86 |
 |    3 |   1 |   3.728 |
 |    4 |   1 |    9.08 |
 |    5 |   1 |   3.237 |
 |    6 |   1 |   2.473 |
 |    7 |   1 |   1.916 |
 |    8 |   1 |   1.988 |
 |    9 |   1 |   3.671 |
 |   10 |   1 |   2.506 |
 |   11 |   1 |    2.55 |
 |   12 |   1 |   2.242 |
 |   13 |   1 |    3.36 |
 |   14 |   1 |   2.785 |
 |   15 |   1 |   2.682 |
 |   16 |   1 |   1.237 |
 |   17 |   1 |   10.46 |
 |   18 |   1 |   3.034 |
 |   19 |   1 |   1.846 |
 |   20 |   1 |   1.528 |
 |  ... | ... |     ... |
 ------------------------
GenFrame has selected 999999/1000000 processed events in 23.4056s, (42724.79 ev/s)


In [11]:
print(pn.FrameGen(evs).limit(int(1E6)).progress(int(1E6)-1).evaluate())

 --------------
 | evt# | cvw |
 --------------
 |    0 |   1 |
 |    1 |   1 |
 |    2 |   1 |
 |    3 |   1 |
 |    4 |   1 |
 |    5 |   1 |
 |    6 |   1 |
 |    7 |   1 |
 |    8 |   1 |
 |    9 |   1 |
 |   10 |   1 |
 |   11 |   1 |
 |   12 |   1 |
 |   13 |   1 |
 |   14 |   1 |
 |   15 |   1 |
 |   16 |   1 |
 |   17 |   1 |
 |   18 |   1 |
 |   19 |   1 |
 |   20 |   1 |
 |  ... | ... |
 --------------
GenFrame has selected 999999/1000000 processed events in 23.2454s, (43019.18 ev/s)


Seems like there are neglible call overheads for the JITted C++ function as a projection-less loop takes the same amount of time. The majority of loop time is I/O of the file.